# Introduction to the Keras Tuner

This tutorial follows through the hyperparameter tuning (hypertuning), working on the Fashion MNIST dataset. There are two kinds of hyperparameters:
1. **Model hyperparameters**, influencing model selection (_e.g._ number / width of hidden layers)
2. **Algorithm hyperparameters**, influencing speed, quality, _etc_. of the algorithm (_e.g._ the learning rate or the number of nearest neighbours for a k Nearest Neighbour classifier).

Let's start by importing the necessary libraries and load and prepare the data.

In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

### Define the model
We define the model by defining a model builder function to define the image classification model.  

In [ ]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model